In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np  
import time
# import statistics
from tqdm import tqdm

In [2]:
# 劣勾配の計算
# (1) |w|の要素を減少順にソートする
# (2) i=1, ..., Kならw(i)に対応するsの要素のs(i)にsign(w(i))を代入し、それ以外は0を代入する
def calculate_subgradient(w, K):
    n = len(w)
    s = np.zeros(n)
    w = np.sort(np.abs(w))[::-1] #(1)
    s[:K] = np.sign(w[:K]) #(2)
    return s

In [8]:
# x = argmin(cx + ρ|z|_1 - zs_z^{t-1})を解く
def solve_argmin_x(C, rho, s_t_minus_1, T, xi, z_t_minus_1, M):
    '''
    K: 供給者数
    J: 需要者数
    n: シナリオ数
    '''

    K, J = C.shape
    n = len(xi)
    # p = int(n * epsilon)

    # zの劣勾配の計算
    # s_t_minus_1 = calculate_subgradient(z_t_minus_1, p)
    
    # モデルの作成
    model = gp.Model("sub_x")
    
    # 変数の定義
    X = model.addMVar((K, J), lb=0.0, vtype=GRB.CONTINUOUS, name="X")

    # 目的関数の定義
    objective = gp.quicksum(C[k, j] * X[k, j] for k in range(K) for j in range(J)) + rho * gp.quicksum(z_t_minus_1[i] for i in range(n))- gp.quicksum(z_t_minus_1[i] * s_t_minus_1[i] for i in range(n))
    model.setObjective(objective, sense=GRB.MINIMIZE)

    # 制約条件の定義
    for i in range(n):
        model.addConstr(T @ X >= (1-z_t_minus_1[i]) * xi[i])
    
    # 供給制約 
    for k in range(K):
        model.addConstr(gp.quicksum(X[k, j] for j in range(J)) <= M[k])
    
    #　時間上限
    model.Params.TimeLimit = 1800

    # 最適化の実行 
    model.optimize()
    
    # 結果の表示
    if model.status == GRB.OPTIMAL:
        optimal_X = np.array([[X[k, j].x for j in range(J)] for k in range(K)]) 
        optimal_obj_value = model.objVal
        return optimal_X, optimal_obj_value
    else:
        print("最適解が見つかりませんでした。")
        return None, None

In [9]:
np.random.seed(42)
z_t_minus_1 = np.random.rand(1000)
s_t_minus_1 = calculate_subgradient(z_t_minus_1, 100)
C = C = np.random.uniform(10, 50, size=(40, 100))
T = np.ones((1, 40))
M = np.array([100] * 40)
xi = []
# making_xi_time_s = time.time()
for _ in range(1000):
    mean = np.random.uniform(0, 10, 100)  # 平均ベクトルをランダムに生成
    variance = np.random.uniform(1, 5, 100)  # 分散ベクトルをランダムに生成
    consumer_demand = np.abs(np.random.normal(mean, np.sqrt(variance), 100))  # 正規分布に従うランダムベクトルを生成
    xi.append(consumer_demand)
xi = np.array(xi)
rho = 10000
X_t, obj_value = solve_argmin_x(C, rho, s_t_minus_1, T, xi, z_t_minus_1, M)
print(X_t.shape)
print(type(X_t))

Set parameter TimeLimit to value 1800
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 100040 rows, 4000 columns and 4004000 nonzeros
Model fingerprint: 0x56f7bbda
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-05, 1e+02]
Presolve removed 99900 rows and 0 columns
Presolve time: 0.46s
Presolved: 140 rows, 4000 columns, 8000 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier performed 0 iterations in 0.46 seconds (1.29 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Iteration    Objective       Primal Inf.    Dual Inf.      Time
     100    4.9156493e+06   0.000000e+00   0.000000e+00      1s

S

In [59]:
def solve_argmin_z(C, rho, s_t_minus_1, T, xi, X_t, M):
    '''
    K: 供給者数
    J: 需要者数
    n: シナリオ数
    '''

    K, J = C.shape
    n = len(xi)
    
    # モデルの作成
    model = gp.Model("sub_x")
    
    # 変数の定義
    z = model.addVars(n, lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name="z")

    # 目的関数の定義
    objective = gp.quicksum(C[k, j] * X_t[k, j] for k in range(K) for j in range(J)) + rho * gp.quicksum(z[i] for i in range(n))- gp.quicksum(z[i] * rho * s_t_minus_1[i] for i in range(n))
    model.setObjective(objective, sense=GRB.MINIMIZE)

    # 制約条件の定義
    for i in range(n):
        for j in range(J):
            model.addConstr(gp.quicksum(T[0, k] * X_t[k, j] for k in range(K)) >= (1-z[i]) * xi[i][j])
    
    # 供給制約 
    for k in range(K):
        model.addConstr(gp.quicksum(X_t[k, j] for j in range(J)) <= M[k])
    
    #　時間上限
    model.Params.TimeLimit = 1800

    # 最適化の実行 
    model.optimize()
    
    # 結果の表示
    if model.status == GRB.OPTIMAL:
        optimal_z = np.array([z[i].x for i in range(n)])
        optimal_obj_value = model.objVal
        return optimal_z, optimal_obj_value
    else:
        print("最適解が見つかりませんでした。")
        return None, None


In [28]:
G = T @ X_t
print(G)
print(G.shape)
print(xi[0].shape)

[[11.92307692 11.24897911 11.16988347 12.47731608 11.16125351 12.28270786
  13.81858059 11.99342848 12.57007603 11.36028832 11.71345263 12.23226861
  11.13459995 12.81449946 11.90680779 13.29221286 11.79223798 10.78209754
  12.6338166  11.7311503  12.50631322 11.87734539 11.06079987 11.93262821
  13.48131832 12.55548703 13.9287078  10.84166562 12.81861206 11.14360442
  12.93173535 13.83331445 10.49839233 11.15198593 14.19539824 12.88317196
  12.2818056  11.6268379  11.41162186 11.26439367 11.76496113 13.47345577
  13.14654829 13.06343104 11.77015187 11.79247063 12.13984649 11.50738988
  12.51150606 11.95876604 12.99771186 10.75367008 11.7768611  12.50491775
  10.53318475 12.79237657 11.69107203 12.08344622 12.23429933 11.64803981
  12.10558949 12.95168038 12.23043691 12.31425097 13.31998637 12.21843845
  12.07926768 11.66764128 12.02988815 11.26760967 13.6987181  13.90535745
  11.68584016 13.10760647 13.41182234 10.64020623 11.18463364 12.82579505
  11.52194091 10.72661208 14.02905472 

In [36]:
z_t = solve_argmin_z(C, rho, s_t_minus_1, T, xi, X_t, M)[0]
print(z_t)

Set parameter TimeLimit to value 1800
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 100040 rows, 1000 columns and 100000 nonzeros
Model fingerprint: 0x1f2285cd
Coefficient statistics:
  Matrix range     [1e-04, 2e+01]
  Objective range  [1e+04, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-04, 1e+02]
Presolve removed 100040 rows and 1000 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.6414401e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.03 seconds (0.03 work units)
Optimal objective  6.641440117e+05
[1.19235395e-01 6.88258337e-02 4.54382204e-02 9.20425915e-03
 6.95139943e-02 0.00000000e+00 5.80836122e-02 1.95896043e-01
 0.00000000e+00 4.92972764e-02 0.00000000e+00 1.03311799e-01
 2.16

In [60]:
def dca0528(T, C, M, xi, epsilon, rho, max_iter=1000):

    '''
    K: 供給者数
    J: 需要者数
    n: シナリオ数
    '''
    np.random.seed(42)

    # 初期値の設定
    f_t_minus_1 = 0
    z_t_minus_1 = np.random.rand(len(xi))

    start = time.time()
    for iteration in range(max_iter):
        s_t_minus_1 = calculate_subgradient(z_t_minus_1, int(len(xi) * epsilon))

        X_t = solve_argmin_x(C, rho, s_t_minus_1, T, xi, z_t_minus_1, M)[0]
        z_t = solve_argmin_z(C, rho, s_t_minus_1, T, xi, X_t, M)[0]
        f_t = np.sum(C * X_t)

        if np.abs(f_t - f_t_minus_1) < 1e-4:
            print(f'{iteration+1}回目の更新で収束しました。')
            break

        f_t_minus_1 = f_t
        z_t_minus_1 = z_t
    end = time.time()

    return X_t, z_t, f_t, end-start
    print("最適解X:", X_t)
    print("最適解z：", z_t)
    print("最適解の目的関数値：", f_t)
    print("処理時間：", end-start, "秒")


In [41]:
# パラメータ
# シードの設定（再現性のため）
np.random.seed(42)

# 供給者数
K = 40
# 消費者数
J = 100
# シナリオ数
n = 1000

T = np.ones((1, K))
# print("T", T)
C = np.random.uniform(10, 50, size=(K, J))
# print("c", C)
M = np.array([100] * K)


# 消費者の需要を表すランダムベクトルの生成
xi = []
# making_xi_time_s = time.time()
for _ in range(n):
    mean = np.random.uniform(0, 10, J)  # 平均ベクトルをランダムに生成
    variance = np.random.uniform(1, 5, J)  # 分散ベクトルをランダムに生成
    consumer_demand = np.abs(np.random.normal(mean, np.sqrt(variance), J))  # 正規分布に従うランダムベクトルを生成
    xi.append(consumer_demand)
# making_xi_time_e = time.time()
# print("xi生成時間：", making_xi_time_e-making_xi_time_s, "秒")
xi = np.array(xi)
# print("xi", xi.shape)

In [61]:
X_t, z_t, f_t, t = dca0528(T, C, M, xi, epsilon=0.1, rho=10000, max_iter=1000)

Set parameter TimeLimit to value 1800
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 100040 rows, 4000 columns and 4004000 nonzeros
Model fingerprint: 0x8b13f6dd
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-05, 1e+02]
Presolve removed 99900 rows and 0 columns
Presolve time: 0.41s
Presolved: 140 rows, 4000 columns, 8000 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier performed 0 iterations in 0.41 seconds (1.29 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Iteration    Objective       Primal Inf.    Dual Inf.      Time
     100    4.9157696e+06   0.000000e+00   0.000000e+00      1s

S

In [62]:
# z_tに含まれる1の数をカウント
print(np.sum(z_t))

155.28078913365022


In [57]:
print("最適解X:", X_t)

最適解X: [[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [11.06442789  0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


In [63]:
print("最適値：", f_t)

最適値： 13251.099056604351


In [64]:
def process_vector(w, K):
    w = np.asarray(w, dtype=float)
    # print("w", w)
    # wを減少順に並べ替える
    sorted_indices = np.argsort(-np.abs(w))
    sorted_w = np.abs(w[sorted_indices])
    # print("sorted indices", sorted_indices)
    # print("sorted_w", sorted_w)

    # w(i)をK番目まで1、それ以外を0にする
    s = np.zeros_like(w)
    s[sorted_indices[:K]] = np.sign(w[sorted_indices[:K]])
    # print("s", s)

    return sorted_w, s

In [66]:
np.random.seed(42)
w = np.random.rand(10)
print(process_vector(w, 3)[1])
print(calculate_subgradient(w, 3))

[0. 1. 1. 0. 0. 0. 0. 1. 0. 0.]
[1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]


In [71]:
def compute_subgradient(w, K):
    n = len(w)
    s = np.zeros(n)
    sorted_indices = np.argsort(np.abs(w))[::-1]  # Sort indices by absolute value in descending order

    for i in range(n):
        if i < K:
            s[sorted_indices[i]] = np.sign(w[sorted_indices[i]])
        else:
            s[sorted_indices[i]] = 0
    
    return s

In [72]:
np.random.seed(42)
w = np.random.rand(10)
print(process_vector(w, 3)[1])
print(calculate_subgradient(w, 3))
print(compute_subgradient(w, 3))

[0. 1. 1. 0. 0. 0. 0. 1. 0. 0.]
[1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 1. 0. 0. 0. 0. 1. 0. 0.]


In [74]:
import numpy as np
print(np.random.rand(1000))

[0.22120944 0.98766801 0.94405934 0.03942681 0.70557517 0.92524832
 0.18057535 0.56794523 0.9154883  0.03394598 0.69742027 0.29734901
 0.9243962  0.97105825 0.94426649 0.47421422 0.86204265 0.8445494
 0.31910047 0.82891547 0.03700763 0.59626988 0.23000884 0.12056689
 0.0769532  0.69628878 0.33987496 0.72476677 0.06535634 0.31529034
 0.53949129 0.79072316 0.3187525  0.62589138 0.88597775 0.61586319
 0.23295947 0.02440078 0.87009887 0.02126941 0.87470167 0.52893713
 0.9390677  0.79878324 0.99793411 0.35071182 0.76718829 0.40193091
 0.47987562 0.62750546 0.87367711 0.98408347 0.76827341 0.41776678
 0.421357   0.7375823  0.23877715 0.11047411 0.35462216 0.28723899
 0.29630812 0.23360775 0.04209319 0.01787393 0.98772239 0.42777313
 0.38432665 0.67964728 0.21825389 0.94996118 0.78634501 0.089411
 0.41758078 0.87911831 0.94473202 0.46740151 0.61341139 0.16703395
 0.99116863 0.2316717  0.94273177 0.64964665 0.60773679 0.51268851
 0.23066981 0.17652803 0.22048621 0.18643826 0.77958447 0.3501252

In [78]:
# np.random.rand(1000)と同じサイズのベクトルを生成
np.random.seed(2)
print(np.random.rand(1000))
print(type(np.random.rand(1000)))

[0.4359949  0.02592623 0.54966248 0.43532239 0.4203678  0.33033482
 0.20464863 0.61927097 0.29965467 0.26682728 0.62113383 0.52914209
 0.13457995 0.51357812 0.18443987 0.78533515 0.85397529 0.49423684
 0.84656149 0.07964548 0.50524609 0.0652865  0.42812233 0.09653092
 0.12715997 0.59674531 0.226012   0.10694568 0.22030621 0.34982629
 0.46778748 0.20174323 0.64040673 0.48306984 0.50523672 0.38689265
 0.79363745 0.58000418 0.1622986  0.70075235 0.96455108 0.50000836
 0.88952006 0.34161365 0.56714413 0.42754596 0.43674726 0.77655918
 0.53560417 0.95374223 0.54420816 0.08209492 0.3663424  0.8508505
 0.40627504 0.02720237 0.24717724 0.06714437 0.99385201 0.97058031
 0.80025835 0.60181712 0.76495986 0.16922545 0.29302323 0.52406688
 0.35662428 0.04567897 0.98315345 0.44135492 0.50400044 0.32354132
 0.25974475 0.38688989 0.8320169  0.73674706 0.37921057 0.01301734
 0.79740494 0.2693888  0.58268489 0.02555094 0.66220202 0.38752343
 0.4970738  0.41490584 0.3508719  0.55097791 0.97291069 0.11277

In [79]:
np.ones(1000)
print(type(np.ones(1000)))

<class 'numpy.ndarray'>


In [87]:
# txtファイルの3の倍数の行を取得
import numpy as np
import os

# ファイルの読み込み
file_path = "time_dca_0531.txt"
# txtファイルの3の倍数の行を取得
# 最後の改行文字を削除
# strからfloatに変換
# リストに格納
with open(file_path, "r") as f:
    lines = f.readlines()
    lines = [float(line.rstrip()) for i, line in enumerate(lines) if i % 3 == 2]
    print(lines)
    print(type(lines))

# with open(file_path, "r") as f:
#     lines = f.readlines()
#     lines = [line for i, line in enumerate(lines) if i % 3 == 2]
#     print(lines)
#     print(type(lines))

[15507.122788568715, 15507.122788568715, 15524.545006714445, 15486.230390073935, 15492.042797165548, 15516.181689203107, 15494.657126302744, 15488.462678724783, 15491.735490272446, 15503.155914970757, 15466.536405367306, 15489.570511729613, 15508.073778857291, 15499.43160675601, 15516.03488677863, 15464.23259456667, 15493.196341988929, 15514.825041136946, 15483.566976950595, 15509.316172192419, 15515.862505246065, 15497.434952016354, 15453.065124804174, 15513.250172009572, 15533.496481630353, 15517.454087551203, 15489.876771066576, 15492.755756832692, 15507.285346339542, 15480.59710832953, 15496.083978170192, 15490.728539695761, 15501.974409780358, 15447.695168045067, 15477.643963928658, 15512.136201282858, 15525.983922076553, 15503.194933967405, 15510.674963887139, 15481.796990791985, 15466.008230912104, 15512.359439694066, 15497.706349004777, 15453.448764092016, 15483.549569007211, 15475.187941322918, 15487.083711065856, 15490.549502511536, 15472.8061233178, 15478.426532506652, 15487

In [90]:
# linesの最小値と要素のインデックスを取得
print(min(lines))
print(lines.index(min(lines)))

15447.695168045067
33


In [91]:
lines[33]

15447.695168045067

In [5]:
import random
import numpy as np

def create_random_array(n, epsilon):
    num_ones = int(n * epsilon)
    num_zeros = n - num_ones
    arr = np.array([1] * num_ones + [0] * num_zeros)
    np.random.shuffle(arr)
    return arr

n = 10
epsilon = 0.3
random_array = create_random_array(n, epsilon)
print(random_array)

[0 1 1 0 0 1 0 0 0 0]


In [3]:
import numpy as np
z_t_minus_1 = np.random.randint(0, 2, 10)
print(z_t_minus_1)

[0 0 1 0 1 0 1 0 1 0]


In [4]:
print(type(z_t_minus_1))

<class 'numpy.ndarray'>


In [6]:
import numpy as np

n = 10
epsilon = 0.3

num_ones = int(n * epsilon)
num_zeros = n - num_ones

arr = np.concatenate((np.ones(num_ones), np.zeros(num_zeros)))

print(arr)

[1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]


In [7]:
# ファイルの読み込み
file_path = "time_dca_0611.txt"
# txtファイルの3の倍数の行を取得
# 最後の改行文字を削除
# strからfloatに変換
# リストに格納
with open(file_path, "r") as f:
    lines = f.readlines()
    lines = [float(line.rstrip()) for i, line in enumerate(lines) if i % 3 == 2]
    print(lines)
    print(type(lines))

# linesの最小値と要素のインデックスを取得
print(min(lines))
print(lines.index(min(lines)))


[15487.52111579453, 15493.251913333257, 15477.741794529718, 15514.792444592744, 15534.124634807646, 15521.065711515155, 15484.554165333957, 15473.898583764572, 15526.6683500323, 15533.029259321525, 15430.256091759173, 15498.262403376537, 15437.295835369183, 15462.365014893787, 15499.603002944341, 15485.005419991297, 15510.7982346328, 15534.623134552903, 15487.911155233965, 15426.969698105224, 15499.673276929501, 15524.260893426934, 15477.243252083852, 15503.04196012493, 15491.339422788173, 15485.412495319462, 15481.29300218139, 15481.254818914995, 15510.213548803247, 15488.43881368246, 15542.600142287161, 15483.597549590404, 15540.00335789369, 15536.33730226029, 15449.344559764912, 15394.57932891781, 15466.89237189769, 15481.881235638728, 15451.031464122536, 15541.480643123814, 15515.158049372441, 15489.586490270796, 15478.060127090012, 15471.54908719106, 15499.943910724565, 15465.369102405733, 15509.49214188755, 15499.314881039316, 15528.603079916742, 15471.696295807324, 15494.1075193

In [8]:
z_t_minus_1

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 0])

In [10]:
for z in z_t_minus_1:
    if z>0:
        print("z>0")
    else:
        print("z<=0")

z<=0
z<=0
z>0
z<=0
z>0
z<=0
z>0
z<=0
z>0
z<=0


In [11]:
# y_t(np.array)の要素はz_t_minus_1の要素が0より大きいければ1, 0以下なら0を代入する
y_t = np.where(z_t_minus_1 > 0, 1, 0)
y_t

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 0])

In [12]:
y_t = np.zeros_like(z_t_minus_1)
y_t

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])